Make predictions by processing videos and giving a video as an output

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial.distance import euclidean

# Load YOLO ONNX model
model = YOLO("C:/Users/aykaq/Downloads/Hornet_detection_20-01/raspb_files/Final_11/weights/best.pt")

# Open the video file
video_path = "C:/Users/aykaq/Downloads/Hornet_detection_20-01/GP047419 4m40 GOED - Trim.MP4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video writer
output_path = "C:/Users/aykaq/Downloads/Hornet_detection_20-01/GP047419 14-02 - Trim.MP4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Store detected hornet colors for re-identification
hornet_colors = {}
next_hornet_id = 0  

def extract_dominant_color(image, box):
    """Extracts dominant color from detected hornet."""
    x1, y1, x2, y2 = map(int, box)
    
    if x2 <= x1 or y2 <= y1:  
        return (60, 255)  # Default to a stable greenish hue in HSV

    hornet_crop = image[y1:y2, x1:x2]

    if hornet_crop.size == 0:
        return (60, 255)  # Return a stable color if no valid pixels

    # Convert to HSV (Hue-Saturation-Value)
    hsv_crop = cv2.cvtColor(hornet_crop, cv2.COLOR_BGR2HSV)

    # Compute histogram to find dominant color
    hist = cv2.calcHist([hsv_crop], [0, 1], None, [180, 256], [0, 180, 0, 256])
    h, s = np.unravel_index(np.argmax(hist), hist.shape)

    return (h, s)  # Keep HSV values for consistency

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    # Run YOLO inference
    results = model.predict(frame, imgsz=640, conf=0.5)
    
    if not results or len(results[0].boxes) == 0:  
        print("⚠️ No detections in this frame.")
        out.write(frame)  # Still write the frame even if no detections
        cv2.imshow("Hornet Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue  

    for result in results:
        boxes = result.boxes.cpu().numpy()

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])

            # Extract dominant color
            detected_color = extract_dominant_color(frame, (x1, y1, x2, y2))

            # Match hornet by color
            matched_id = None

            for hornet_id, stored_color in hornet_colors.items():
                if euclidean(detected_color, stored_color) < 20:  
                    matched_id = hornet_id
                    break

            # Assign new ID if no match
            if matched_id is None:
                matched_id = next_hornet_id
                hornet_colors[matched_id] = detected_color
                next_hornet_id += 1

            print(f"✅ Detected Hornet {matched_id} | Color: {detected_color}")

            # Draw bounding box & label
            box_color = (int(detected_color[0] * 1.4), int(detected_color[1] * 1.4), 255)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
            label = f"Hornet {matched_id}: {confidence:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

            # Save frame when a hornet is detected
            frame_name = f"/home/on8ei/BeeSafe/detections/hornetl_{matched_id}.jpg"
            cv2.imwrite(frame_name, frame)

    # Write processed frame
    out.write(frame)

    # Show detections
    cv2.imshow("Hornet Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Detection complete. Video saved as {output_path}")

Change the format to ONNX to export the model to the raspberry pi

In [ ]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("C:/Users/aykaq/Downloads/Hornet_detection_20-01/raspb_files/Final_11/weights/best.pt")  # Make sure to use your trained model

# Export the model to NCNN format
model.export(format="onnx")  # Creates 'best_ncnn_model'


Connect to the camera  (change the path to do it on py)

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLO ONNX model
model = YOLO("/home/on8ei/BeeSafe/best.onnx")

# Open the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not open camera.")
    exit()

frame_count = 0


while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Error: Failed to capture frame.")
        break

    # Run YOLO inference
    results = model.predict(frame, imgsz=640, conf=0.5)  # Lower confidence to detect more

    hornet_detected = False  # Track if hornet is found in this frame

    print(f"📢 Raw Model Output: {results}")  # Debugging: Print detection output

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])

            print(f"🟢 Detection: x1={x1}, y1={y1}, x2={x2}, y2={y2}, conf={confidence}, class={class_id}")

            if class_id == 0 and confidence > 0.1:  # Lowered confidence
                hornet_detected = True
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                label = f"Hornet: {confidence:.2f}"
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    if hornet_detected:
        frame_path = f"/home/on8ei/BeeSafe/detected_frame_{frame_count}.jpg"
        cv2.imwrite(frame_path, frame)

        print(f"✅ Hornet detected! Frame saved: {frame_path}")
        frame_count += 1

    # **REMOVE `cv2.imshow()` due to SSH issues**
    cv2.imshow("Hornet Detection", frame)  

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("📌 Camera stream stopped.")


Make predictions live with camera connected

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial.distance import euclidean

# Load YOLO ONNX model
model = YOLO("/home/on8ei/BeeSafe/best.onnx")  # Load ONNX model

# Open camera (use 0, 1, or 2 depending on your camera index)
cap = cv2.VideoCapture(0)  # Change to 1 or 2 if needed

# Set camera resolution (Optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Storage for hornet colors & re-identification
hornet_colors = {}
next_hornet_id = 0  # ID counter for new hornets

def extract_dominant_color(image, box):
    """Extracts dominant color from the detected hornet region."""
    x1, y1, x2, y2 = map(int, box)

    # Ensure the bounding box is valid
    if x2 <= x1 or y2 <= y1:
        return None  

    hornet_crop = image[y1:y2, x1:x2]

    # If crop is empty, return a default color
    if hornet_crop.size == 0:
        return None  

    # Convert to HSV color space
    hsv_crop = cv2.cvtColor(hornet_crop, cv2.COLOR_BGR2HSV)

    # Compute **mean HSV** (instead of just histogram max)
    mean_color = np.mean(hsv_crop, axis=(0, 1))

    return (int(mean_color[0]), int(mean_color[1]))  # Keep Hue & Saturation only

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Camera not detected. Check connection.")
        break  

    # Run YOLO inference
    results = model.predict(frame, imgsz=640, conf=0.1)

    for result in results:
        boxes = result.boxes.cpu().numpy()

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])

            # Extract dominant color of the detected hornet
            detected_color = extract_dominant_color(frame, (x1, y1, x2, y2))

            if detected_color:
                matched_id = None

                # **Re-identify hornets based on color similarity**
                for hornet_id, stored_color in hornet_colors.items():
                    if euclidean(detected_color, stored_color) < 30:  # Looser threshold
                        matched_id = hornet_id
                        break

                # If no match, assign a new ID
                if matched_id is None:
                    matched_id = next_hornet_id
                    hornet_colors[matched_id] = detected_color
                    next_hornet_id += 1

                print(f"✅ Detected Hornet {matched_id} | Color: {detected_color}")

                # Convert HSV to BGR for display
                box_color = cv2.cvtColor(
                    np.uint8([[[detected_color[0], detected_color[1], 255]]]),
                    cv2.COLOR_HSV2BGR)[0][0]
                box_color = tuple(int(c) for c in box_color)

                # Draw bounding box & label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
                label = f"Hornet {matched_id}: {confidence:.2f}"
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

                # Save frame when a hornet is detected
                frame_name = f"/home/on8ei/BeeSafe/detections/hornet_{matched_id}.jpg"
                cv2.imwrite(frame_name, frame)

      # **Save frame only if a hornet was detected**

    # **Show detections while running**
    cv2.imshow("Hornet Detection Live", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

print("✅ Live detection stopped. All detected hornet images saved.")


When a hornet is detected the led goes on

In [ ]:
import cv2
import numpy as np
import RPi.GPIO as GPIO
import time
from ultralytics import YOLO
from scipy.spatial.distance import euclidean

# **Relay GPIO Setup**
RELAY_PIN = 17  # Adjust to your GPIO pin
GPIO.setmode(GPIO.BCM)
GPIO.setup(RELAY_PIN, GPIO.OUT)
GPIO.output(RELAY_PIN, GPIO.LOW)  # Ensure relay is off at start

# Load YOLO ONNX model
model = YOLO("/home/on8ei/BeeSafe/Final_11/weights/best.pt")

# Open the video file
video_path = "/home/on8ei/BeeSafe/GOPR7413 - Trim.MP4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video writer
output_path = "/home/on8ei/BeeSafe/hornet-14-02 captured.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Store detected hornet colors for re-identification
hornet_colors = {}
next_hornet_id = 0  

# Function to extract dominant color
def extract_dominant_color(image, box):
    x1, y1, x2, y2 = map(int, box)
    
    if x2 <= x1 or y2 <= y1:  
        return (60, 255)  # Default greenish hue in HSV

    hornet_crop = image[y1:y2, x1:x2]

    if hornet_crop.size == 0:
        return (60, 255)  

    hsv_crop = cv2.cvtColor(hornet_crop, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_crop], [0, 1], None, [180, 256], [0, 180, 0, 256])
    h, s = np.unravel_index(np.argmax(hist), hist.shape)

    return (h, s)

# Main loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    # Run YOLO inference
    results = model.predict(frame, imgsz=640, conf=0.5)
    
    hornet_detected = False  # Track if a hornet is found

    if results and len(results[0].boxes) > 0:  
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                confidence = float(box.conf[0])

                # Extract dominant color
                detected_color = extract_dominant_color(frame, (x1, y1, x2, y2))

                # Match hornet by color
                matched_id = None
                for hornet_id, stored_color in hornet_colors.items():
                    if euclidean(detected_color, stored_color) < 20:  
                        matched_id = hornet_id
                        break

                if matched_id is None:
                    matched_id = next_hornet_id
                    hornet_colors[matched_id] = detected_color
                    next_hornet_id += 1

                print(f"✅ Detected Hornet {matched_id} | Color: {detected_color}")

                # Ensure color values are integers
                box_color = tuple(map(int, (detected_color[0] * 1.4, detected_color[1] * 1.4, 255)))

                # Draw bounding box & label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
                label = f"Hornet {matched_id}: {confidence:.2f}"
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

                # Save frame when a hornet is detected
                frame_name = f"/home/on8ei/BeeSafe/detections/hornet_{matched_id}.jpg"
                cv2.imwrite(frame_name, frame)

                hornet_detected = True  # Hornet found

    # **Trigger relay if a hornet is detected**
    if hornet_detected:
        GPIO.output(RELAY_PIN, GPIO.HIGH)  # Turn ON light
        print("🔴 LIGHT ON: Hornet detected!")
    else:
        GPIO.output(RELAY_PIN, GPIO.LOW)   # Turn OFF light
        print("🟢 LIGHT OFF: No hornets detected.")

    # Write processed frame
    out.write(frame)

    # Show detections
    cv2.imshow("Hornet Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
GPIO.cleanup()  # Cleanup GPIO

print(f"✅ Detection complete. Video saved as {output_path}")
